# Convert PDF to JPG for faster parsing

In [ ]:
import os
from pdf2image import convert_from_path

# Input & output folders
INPUT_DIR = "pdf_folder"
OUTPUT_DIR = "jpg_output"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Loop through all PDF files in folder
for file in os.listdir(INPUT_DIR):
    if file.lower().endswith(".pdf"):
        pdf_path = os.path.join(INPUT_DIR, file)
        base_name = os.path.splitext(file)[0]

        # Convert PDF to images
        images = convert_from_path(pdf_path)

        # Save each page as JPG with page index
        for i, img in enumerate(images, start=1):
            out_file = f"{base_name}_page{i}.jpg"
            img.save(os.path.join(OUTPUT_DIR, out_file), "JPEG")

        print(f"✅ Saved {len(images)} pages from {file}")


# Init

In [5]:
from __future__ import annotations
import os
import glob
import pandas as pd
from datetime import datetime
from typing import List, Dict, Any

import gspread
from agentic_doc.parse import parse
from pydantic import BaseModel, Field
from agentic_doc.connectors import LocalConnectorConfig
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build

def get_google_credentials():
    """Get Google credentials from Streamlit secrets or local file."""
    cred_path = "suksiri-purchase-test-0f09e84df6dd.json"
    if os.path.exists(cred_path):
        return Credentials.from_service_account_file(
            cred_path,
            scopes=["https://www.googleapis.com/auth/spreadsheets"]
        )
    else:
        raise FileNotFoundError(f"Credentials file not found: {cred_path}")

def get_spreadsheet_id():
    """Get spreadsheet ID from secrets or use default."""
    return "17chQLsKcpyZNnJyw8Ads-WRz45kNvI1AbvwsdIlcXqs"


############################### Store_list ##############################
def get_store_list():
    """Fetch and process store list from Google Sheets."""

    # Get credentials and spreadsheet ID
    creds = get_google_credentials()
    spreadsheet_id = get_spreadsheet_id()
    
    client = gspread.authorize(creds)
    service = build("sheets", "v4", credentials=creds)

    sheets = client.open_by_key(spreadsheet_id)
    store_data = sheets.worksheet("ข้อมูลร้านค้า")
    sheet = sheets.worksheet("รายการสินค้า")

    table_range = "ข้อมูลร้านค้า!B2:G"  

    result = service.spreadsheets().values().get(
        spreadsheetId=spreadsheet_id,
        range=table_range
    ).execute()

    values = result.get("values", [])
    max_len = len(values[0])
    normalized = [row + [""] * (max_len - len(row)) for row in values[1:]]

    # Convert to DataFrame
    df_table2 = pd.DataFrame(normalized, columns=values[0])  # first row is header

    # --- Keep only rows where 'ร้านค้า' is not None or empty ---
    df_filtered = df_table2[df_table2['ร้านค้า'].notna() & (df_table2['ร้านค้า'] != '')].copy()

    # --- Convert 'ยังไม่รวม VAT' from string 'TRUE'/'FALSE' to boolean ---
    df_filtered['ยังไม่รวม VAT'] = df_filtered['ยังไม่รวม VAT'].map(lambda x: True if str(x).upper() == 'TRUE' else False)

    unique_store_list = df_filtered['ร้านค้า'].unique().tolist()

    return unique_store_list, df_filtered
    
 
############################### Build_Rows ##############################
def build_rows(fields):
    metadata = {
        "วันเดือนปี": fields.documentInfo.documentDate,
        "ร้านค้า": fields.companyInfo.companyName,
        "เลขกำกับ": fields.documentInfo.documentNumber,
        # "taxId": fields.companyInfo.taxId,
        # "customerName": fields.customerInfo.customerName,
        # "grossAmount": fields.totals.grossAmount,
        # "netAmount": fields.totals.netAmount,
        # "vat": fields.totals.vat,
        # "grandTotal": fields.totals.grandTotal,
    }
    rows = []
    # Handle items
    for i, item in enumerate(fields.items):
        row = metadata.copy()
        row["รายการสินค้า"] = item.description
        row["จำนวน"] = float(item.quantity.replace(",", ""))
        row["หน่วย"] = item.unitName
        row["ราคาต่อหน่วย"] = float(item.unitPrice.replace(",", ""))
        
        # set defaults to empty cells
        row["ลดราคา(%)"], row["ลดราคา(บาท)"] = "", ""
        if item.discountType == "บาท":
            row["ลดราคา(บาท)"] = float(item.discount.strip('฿').replace(",", ""))
        elif item.discountType == "Percentage":
            row["ลดราคา(%)"] = float(item.discount.strip('%'))

        rows.append(row)
    
    return rows

############################### update_sheet ##############################
def update_sheet(rows):
    
    # Get credentials and spreadsheet ID
    creds = get_google_credentials()
    spreadsheet_id = get_spreadsheet_id()
    client = gspread.authorize(creds)
    service = build("sheets", "v4", credentials=creds)

    sheets = client.open_by_key(spreadsheet_id)
    store_data = sheets.worksheet("ข้อมูลร้านค้า")
    sheet = sheets.worksheet("รายการสินค้า")

    df = pd.DataFrame(rows)
    values = df.values.tolist()
    
    start_row = len(sheet.get_all_values()) + 1  # +1 because Sheets are 1-indexed
    for i, row in enumerate(values):
        current_row = start_row + i

        # Column J: ยอดเงิน
        row.append("=Transactions_2[จำนวน]*Transactions_2[ราคาต่อหน่วย]")

        # Column K: ยอดเงินหลังลดราคา
        row.append("=IF(Transactions_2[ลดราคา(%)], Transactions_2[ยอดเงิน]*(1-Transactions_2[ลดราคา(%)]/100), Transactions_2[ยอดเงิน]-Transactions_2[ลดราคา(บาท)])")

        # Column L: ยอดรวมต่อรายการ
        row.append(f"=SUMIF(Transactions_2[เลขกำกับ], C{current_row}, Transactions_2[ยอดเงินหลังลดราคา])")

        # Column M: ยอดรวมหลังภาษี
        row.append(
            f"=IF(XLOOKUP(B{current_row}, Table2[ร้านค้า], Table2[ยังไม่รวม VAT], FALSE), $L{current_row}*1.07, $L{current_row})"
        )

    # Append all rows to the sheet
    sheet.append_rows(values, value_input_option="USER_ENTERED")

    return pd.DataFrame(values)

# ModelClass

In [6]:
unique_store_list, _ = get_store_list()
####################### ExtractedDocumentFieldsSchema Class ##############################
class CompanyInfo(BaseModel):
    companyName: str = Field(
        ...,
        description=f'Identify the official name of the seller/supplier company as stated in the document. Compare it to the following list of known companies: [{", ".join(unique_store_list)}]. Return the closest match, ignoring common words like ["ห้างหุ้นส่วนจำกัด", "บริษัท", "จำกัด", "บจก.", "หจก.", "จํากัด", "ก้าวไกล"] for matching purposes. If no sufficiently similar match is found, retain the original text from the document.',
        title='Company Name',
    )
    taxId: str = Field(
        ...,
        description="The seller/supplier company's tax identification number.",
        title='Tax Identification Number',
    )

class CustomerInfo(BaseModel):
    customerName: str = Field(
        ..., description='The name of the customer or recipient.', title='Customer Name'
    )
    
class DocumentInfo(BaseModel):
    documentNumber: str = Field(
        ...,
        description='Unique identifier or reference number(เลขที่กำกับ) for the document.',
        title='Document Number',
    )
    documentDate: str = Field(
        ..., description='Date the document was issued with Year formatted in คริสต์ศักราช (ค.ศ.)/AD (Anno Domini) if it was initially written in the format of พุทธศักราช (พ.ศ.)/BE (Buddhist Era)', title='Document Date'
    )
    
class Item(BaseModel):
    description: str = Field(
        ..., description='Description of the item or service.', title='Description'
    )
    quantity: str = Field(..., description='Quantity of the item.', title='Quantity')
    unitPrice: str = Field(
        ..., description='Price per unit of the item.', title='Unit Price'
    )
    unitName: str = Field(..., description='Unit of measurement for the item, for example, meter, Pcs, ea, kg, box, อัน, ใบ, เส้น, ท่อน, ตัว, กระป๋อง, หลอด, ม้วน', title='Unit Name')
    amount: str = Field(..., description='Total amount for the item.', title='Amount')
    discount: str = Field(..., description='Discount applied to the item.', title='Item Discount')
    discountType: str = Field(..., description='Type of discount applied in Thai Baht or in Percentage or No Discount', title='Discount Type')

class Totals(BaseModel):
    grossAmount: str = Field(
        ...,
        description='Total gross amount before discounts and taxes.',
        title='Gross Amount',
    )
    netAmount: str = Field(
        ..., description='Net amount after discounts.', title='Net Amount'
    )
    vat: str = Field(..., description='Value-added tax amount.', title='VAT')
    grandTotal: str = Field(
        ...,
        description='Total amount payable including all taxes.',
        title='Grand Total',
    )

class ExtractedDocumentFieldsSchema(BaseModel):
    
    companyInfo: CompanyInfo = Field(
        ...,
        description='Key company details from headers and form fields.',
        title='Company Information',
    )
    customerInfo: CustomerInfo = Field(
        ...,
        description='Details about the customer or recipient of the invoice/delivery.',
        title='Customer Information',
    )
    documentInfo: DocumentInfo = Field(
        ...,
        description='Key identifiers and dates for the document.',
        title='Document Information',
    )
    items: List[Item] = Field(
        ...,
        description='List of items, products, or services from the main table(s) in the document.',
        title='Itemized Table',
    )
    totals: Totals = Field(
        ...,
        description='Summary of financial totals from the document.',
        title='Totals and Summary',
    )
    

file_cache is only supported with oauth2client<4.0.0 (__init__.py:49)


# Main Extraction

In [7]:

# Configuration
BASE_DIR = "drive-download-20250912T082149Z-1-001"
SUPPORTED_EXTENSIONS = ["*.jpg"]
OUTPUT_DIR = "batch_processing_results"

os.makedirs(OUTPUT_DIR, exist_ok=True)

# Find directories with files
subdirs = []
for root, dirs, files in os.walk(BASE_DIR):
    if any(any(file.lower().endswith(ext.replace("*", "").lower()) for ext in SUPPORTED_EXTENSIONS) for file in files):
        subdirs.append(root)

all_rows = []

# Process each directory
for subdir in sorted(subdirs)[:1]:
    print('='*60)
    print(subdir)
    
    config = LocalConnectorConfig()
    
    
    
    for ext in SUPPORTED_EXTENSIONS:
        all_rows = []
        
        print(f"Files with {ext}...")
        extraction_results = parse(
            config,
            connector_path=subdir,
            connector_pattern=ext,
            extraction_model=ExtractedDocumentFieldsSchema
        )
        
        for result in extraction_results:
            try:
                fields = result.extraction
                rows = build_rows(fields)
                all_rows.extend(rows)  # extend instead of append
            except Exception:
                continue
                


        df = pd.DataFrame(all_rows)

        if df.empty:
            print('df.empty')
        else:
            # Save to CSV
            CSV_OUTPUT = f"{subdir.split('/')[-1]}_{ext.split('*.')[-1]}_extracted_data_{datetime.now().strftime('%Y%m%d%H%M%S')}.csv"
            csv_path = os.path.join(OUTPUT_DIR, CSV_OUTPUT)
            df.to_csv(csv_path, index=False, encoding='utf-8')
            
            # Convert to list of dicts and upload to Google Sheets
            rows = df.to_dict('records')
            
            # Upload to GG Sheets
            # update_sheet(rows)
    
    print('='*60)
    print(f'Saved {subdir}')

drive-download-20250912T082149Z-1-001/8สค68
Files with *.jpg...
2025-09-16 14:47:40 [info   ] API key is valid.              [agentic_doc.utils] (utils.py:42)
2025-09-16 14:47:40 [info   ] Parsing 9 documents            [agentic_doc.parse] (parse.py:280)


Parsing documents:   0%|          | 0/9 [00:00<?, ?it/s]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:47:59 [info   ] Time taken to successfully parse a document chunk: 19.37 seconds [agentic_doc.parse] (parse.py:827)


Parsing documents:  11%|█         | 1/9 [00:19<02:34, 19.31s/it]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:48:13 [info   ] Time taken to successfully parse a document chunk: 33.38 seconds [agentic_doc.parse] (parse.py:827)
HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:48:19 [info   ] Time taken to successfully parse a document chunk: 38.98 seconds [agentic_doc.parse] (parse.py:827)
HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:48:19 [info   ] Time taken to successfully parse a document chunk: 39.07 seconds [agentic_doc.parse] (parse.py:827)


...

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:49:04 [info   ] Time taken to successfully parse a document chunk: 44.43 seconds [agentic_doc.parse] (parse.py:827)


Parsing documents:  89%|████████▉ | 8/9 [01:23<00:09,  9.47s/it]

HTTP Request: POST https://api.va.landing.ai/v1/tools/agentic-document-analysis "HTTP/1.1 200 OK" (_client.py:1025)
2025-09-16 14:49:06 [info   ] Time taken to successfully parse a document chunk: 44.05 seconds [agentic_doc.parse] (parse.py:827)


Parsing documents: 100%|██████████| 9/9 [01:26<00:00,  9.57s/it]

Saved drive-download-20250912T082149Z-1-001/8สค68


# Export to GG Sheets

In [8]:
import pandas as pd
import os 

OUTPUT_DIR = 'batch_processing_results'
df_list = []

# for file in sorted(os.listdir(OUTPUT_DIR)):
#     if file.endswith('.csv') and file[0] == '8':
#         print(file)
file = '8สค68_jpg_extracted_data_20250916214906.csv'
df_temp = pd.read_csv(os.path.join(OUTPUT_DIR, file))
df_temp = df_temp.fillna("")  # replace NaN with blank
df_list.append(df_temp)

df = pd.concat(df_list, ignore_index=True)
df.sort_values(by=['วันเดือนปี', 'ร้านค้า', 'เลขกำกับ'], inplace=True)
rows = df.to_dict('records')

update_sheet(rows)

df.shape


file_cache is only supported with oauth2client<4.0.0 (__init__.py:49)


(18, 9)